# Import Modules 

In [242]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [243]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display,clear_output

# Prepare Dataset

In [244]:
df = pd.read_csv("Dataset.csv",parse_dates=['Date'])

df.columns=['date','well','oilrate','waterrate','sand']
dt = df['date'].dt
df.dropna(inplace=True)

df.head()

,date,well,oilrate,waterrate,sand
0,1987-12-01,01-02,432.0,0.0,CHANNEL
1,1988-01-01,01-02,518.0,0.0,CHANNEL
2,1988-02-01,01-02,452.0,0.0,CHANNEL
3,1988-03-01,01-02,446.0,0.0,CHANNEL
4,1988-04-01,01-02,450.0,0.0,CHANNEL


# Calculate MHI and Cum MHI

In [265]:
wells = df['well'].unique()
sands = df['sand'].unique()

In [264]:
ave_grouped_rate = df.groupby(['sand','date']).mean()
max_grouped_rate = df.groupby(['sand','date']).max()
min_grouped_rate = df.groupby(['sand','date']).min()

In [247]:
hi_types = {
    'hi_oil':'oilrate',
    'hi_water':'waterrate'
}

def calc_HI(row):
    if row[hi_types[key]] > 0:
        return (row[hi_types[key]]-ave_grouped_rate.loc[(row['sand'],row['date']),hi_types[key]])/\
             (max_grouped_rate.loc[(row['sand'],row['date']),hi_types[key]]-min_grouped_rate.loc[(row['sand'],row['date']),hi_types[key]])
    else:
        return 0
    
for key in hi_types.keys():
    df[key] = df.apply(lambda row: calc_HI(row),axis=1)
    df['cum_'+ key] = df.groupby('well').cumsum()[key]

C:\Users\ANaufal\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in double_scalars
  if __name__ == '__main__':


# Classify Wells to Quadrants

In [248]:
def classify(df):
    w_dict = {}
    
    for w in df['well'].unique():
        last_cum_hi_oil = df[(df['well']==w) & (df['date']==df[df['well']==w]['date'].max())]['cum_hi_oil'].iloc[0]
        last_cum_hi_water = df[(df['well']==w) & (df['date']==df[df['well']==w]['date'].max())]['cum_hi_water'].iloc[0]
        
        if last_cum_hi_oil > 0 and last_cum_hi_water > 0:
            w_dict[w] = 'High Oil and High Water'
        elif last_cum_hi_oil <= 0 and last_cum_hi_water > 0:
            w_dict[w] = 'Low Oil and High Water'
        elif last_cum_hi_oil > 0 and last_cum_hi_water <= 0:
            w_dict[w] = 'High Oil and Low Water'
        elif last_cum_hi_oil <= 0 and last_cum_hi_water <= 0:
            w_dict[w] = 'Low Oil and Low Water'
    
    return df['well'].map(w_dict)

df['class'] = classify(df)

# Plot MHI Scatter Plot

In [266]:
items = [widgets.Button(description=w) for w in df['sand'].unique()]
display(widgets.HBox([items[0],items[1]]))

fmt='%Y-%m-%d'
date_range = pd.date_range(start=df[df['sand']=='CHANNEL']['date'].min(),end=df[df['sand']=='CHANNEL']['date'].max(),freq='MS')
options = [(item.strftime(fmt),item) for item in date_range]

slider = widgets.SelectionSlider(
    description='Date',
    options=options,
    continuous_update=False
) 

def plot(sand,date):
    clear_output(wait=True)
    plt.figure(figsize=(10,10))
    plt.xlim(-30,70)
    plt.ylim(-30,130)
    plt.axvline(0,color='black')
    plt.axhline(0,color='black')
    hue_dict = {
        'High Oil and Low Water': 'g',
        'Low Oil and High Water': 'brown',
        'High Oil and High Water': 'b',
        'Low Oil and Low Water': 'r'
    }
    sns.scatterplot('cum_hi_oil','cum_hi_water',data=df[(df['sand']==sand)&(df['date']==date)],
                    hue='class', palette=hue_dict,s=50)
    plt.legend(loc=1)

def scatter(b):
    clear_output(wait=True)
    display(widgets.HBox([items[0],items[1]]))
    interact(plot,date=slider,sand=fixed(b.description))

    
items[0].on_click(scatter)
items[1].on_click(scatter)


interactive(children=(SelectionSlider(continuous_update=False, description='Date', options=(('1978-11-01', Tim…